# Домашнее задание: Спекулятивное декодирование и архитектура Qwen

**Курс:** NLP-2

## Описание задания

В этом домашнем задании мы не будем заниматься обучением моделей с нуля. Вместо этого мы сфокусируемся на **Inference Engineering** — области, которая находится на стыке разработки и исследований и занимается оптимизацией и ускорением работы уже обученных больших языковых моделей (LLM).

Мы реализуем с нуля метод **Speculative Decoding** — один из самых популярных алгоритмических подходов, позволяющий ускорить генерацию текста в 1.5-2.5 раза практически без потери качества. Этот метод используется для оптимизации инференса таких моделей, как Llama, Mixtral и других.

### План работы:
1. **Подготовка окружения**: Настроим среду и загрузим модели.
2. **Реализация Draft модели**: Мы вручную, блок за блоком, соберем архитектуру модели `Qwen2.5-0.5B` (`NanoQwen`). Это позволит нам досконально понять устройство современных LLM, включая `RMSNorm`, `RoPE` и `SwiGLU`.
3. **Реализация цикла спекуляции**: Напишем основной алгоритм, в котором Draft модель быстро генерирует черновик, а Target модель его верифицирует.
4. **Бенчмарк**: Проведем замеры и оценим реальное ускорение, которое дает наш метод.

## Шаг 1: Настройка окружения

In [1]:
!pip install -q transformers accelerate safetensors sentencepiece

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from safetensors.torch import load_file
from huggingface_hub import hf_hub_download
import time
import gc
import json
import numpy as np
from tqdm.auto import tqdm
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, set_peft_model_state_dict
import os
from datasets import load_dataset, Dataset
import random
from torch.utils.data import DataLoader


OUTPUT_DIR = "./lora_draft_adapter"
# Фиксируем seed для воспроизводимости
torch.manual_seed(42)
np.random.seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


### Загрузка моделей

В качестве целевой модели (`Target`) мы будем использовать `Qwen/Qwen2.5-1.5B-Instruct`. В качестве черновой модели (`Draft`) — `Qwen/Qwen2.5-0.5B-Instruct`. Мы загрузим `Target` с помощью стандартного класса `AutoModelForCausalLM` из `transformers`, а вот `Draft` модель соберем вручную.

In [3]:
TARGET_ID = "Qwen/Qwen2.5-1.5B-Instruct"
DRAFT_ID = "Qwen/Qwen2.5-0.5B-Instruct"

print("Загрузка токенизатора...")
tokenizer = AutoTokenizer.from_pretrained(TARGET_ID)

print("Загрузка Target модели (через AutoModelForCausalLM)...")
# Используем torch_dtype=torch.float16 для экономии VRAM и attn_implementation="sdpa" для использования Flash Attention
target_model = AutoModelForCausalLM.from_pretrained(
    TARGET_ID,
    torch_dtype=torch.float16,
    device_map="auto",
    attn_implementation="sdpa"
)
target_model.eval()
print("Модель Target загружена.")

Загрузка токенизатора...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Модель Target загружена.


## Шаг 2: Собираем Draft модель "NanoQwen" (4 балла)

На этом шаге мы не будем использовать `AutoModel`, а соберем модель самостоятельно из отдельных модулей. Это ключевая часть задания, которая поможет понять, как современные трансформеры устроены "под капотом".

Сначала загрузим только конфигурацию `Draft` модели, из которой мы будем брать все параметры архитектуры (размер скрытого слоя, количество голов и т.д.).

In [ ]:
draft_config = AutoConfig.from_pretrained(DRAFT_ID)
print(f"Конфигурация Draft модели:")
print(f"  - Размер скрытого слоя (hidden_size): {draft_config.hidden_size}")
print(f"  - Количество слоев (num_hidden_layers): {draft_config.num_hidden_layers}")
print(f"  - Количество голов внимания (num_attention_heads): {draft_config.num_attention_heads}")

### Задание 2.1: RMSNorm

Современные модели, такие как Llama и Qwen, используют **Root Mean Square Layer Normalization (RMSNorm)** вместо классического `LayerNorm`. `RMSNorm` проще и вычислительно эффективнее, так как оперирует только масштабированием на основе среднеквадратичного значения и не использует дополнительный сдвиг (bias).

Формула `RMSNorm` для вектора активаций $\mathbf{x}$:
$$ \text{RMS}(\mathbf{x}) = \sqrt{\frac{1}{n} \sum_{i=1}^{n} x_i^2} $$
$$ \text{RMSNorm}(\mathbf{x}) = \frac{\mathbf{x}}{\text{RMS}(\mathbf{x}) + \epsilon} \cdot \mathbf{w} $$
где $\mathbf{w}$ — обучаемый весовой вектор (гейт), а $\epsilon$ — малая константа для численной стабильности.

**Важно**: При вычислениях в `float16`, промежуточный расчет `variance` (степень `pow(2)`) может привести к переполнению. Поэтому стандартная практика — временно повышать тип данных до `float32` для этого расчета.

**Задание**: Реализуйте `forward` для `QwenRMSNorm`.

**Полезные ссылки**:
- [Root Mean Square Layer Normalization (Zhang and Sennrich, 2019)](https://arxiv.org/abs/1910.07467)

In [ ]:
class QwenRMSNorm(nn.Module):
    """
    Реализация Root Mean Square Layer Normalization.

    Аргументы:
        hidden_size (int): Размер скрытого слоя.
        eps (float): Малая константа для предотвращения деления на ноль.
    """
    def __init__(self, hidden_size, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        # 1. Запомните исходный тип данных
        input_dtype = hidden_states.dtype

        # 2. Переведите hidden_states в float32 для стабильных вычислений
        hidden_states_fp32 = hidden_states.to(torch.float32)

        # 3. Вычислите variance: среднее от квадратов элементов по последней оси
        variance = hidden_states_fp32.pow(2).mean(-1, keepdim=True)

        # 4. Нормализуйте hidden_states
        hidden_states_normalized = hidden_states_fp32 * torch.rsqrt(variance + self.variance_epsilon)

        # 5. Умножьте на обучаемый вес и верните результат в исходном типе данных
        result = self.weight * hidden_states_normalized
        return result.to(input_dtype)


In [ ]:
print("--- Запуск тестов для RMSNorm ---")
try:
    hidden_size = 128
    norm_layer = QwenRMSNorm(hidden_size).to(device).half()

    # Тест 1: Проверка размерности
    dummy_input = torch.randn(2, 10, hidden_size, device=device).half()
    output = norm_layer(dummy_input)
    assert output.shape == dummy_input.shape, f"Ошибка размерности: ожидалось {dummy_input.shape}, получено {output.shape}"
    print("✅ [1/3] Тест на размерность пройден.")

    # Тест 2: Проверка типа данных
    assert output.dtype == torch.float16, f"Ошибка типа данных: ожидалось float16, получено {output.dtype}"
    print("✅ [2/3] Тест на тип данных пройден.")

    # Тест 3: Проверка на NaN
    assert not torch.isnan(output).any(), "В выходе RMSNorm обнаружены NaN значения."
    print("✅ [3/3] Тест на NaN пройден.")

    print("\n🎉 Все тесты для RMSNorm пройдены!")

except Exception as e:
    print(f"❌ Тест RMSNorm провален: {e}")

### Задание 2.2: Rotary Positional Embeddings (RoPE)

`RoPE` — это элегантный способ внедрения позиционной информации, который вместо добавления векторов (как в `sin/cos embeddings`) "вращает" векторы запросов (`Query`) и ключей (`Key`) на угол, зависящий от их позиции.

#### Два подхода к реализации
Существует два эквивалентных способа реализации этого вращения.

1. **Разбиение на пары (Pairwise Rotation)**. Этот метод мы рассматривали на лекции. Вектор признаков $\mathbf{x} = (x_1, x_2, \dots, x_d)$ рассматривается как набор двумерных векторов $(x_{2i-1}, x_{2i})$. Каждый такой вектор вращается в 2D-плоскости:
$$
\begin{pmatrix} x'_{2i-1} \\ x'_{2i} \end{pmatrix} =
\begin{pmatrix} \cos(m\theta_i) & -\sin(m\theta_i) \\ \sin(m\theta_i) & \cos(m\theta_i) \end{pmatrix}
\begin{pmatrix} x_{2i-1} \\ x_{2i} \end{pmatrix}
$$
где $m$ — позиция токена, а $\theta_i$ — частота вращения.

2. **Вращение через половины (`rotate_half`)**. Этот метод используется в реализациях Llama и Qwen, и именно его мы будем использовать. Здесь вектор $\mathbf{x}$ делится на две половины: $\mathbf{x}_1 = (x_1, \dots, x_{d/2})$ и $\mathbf{x}_2 = (x_{d/2+1}, \dots, x_d)$. Вращение реализуется следующим образом:
$$ \mathbf{x}'_m = \mathbf{x}_m \cos(m\theta) + \text{rotate\_half}(\mathbf{x}_m) \sin(m\theta) $$
где операция `rotate_half` преобразует вектор $\mathbf{x} = (\mathbf{x}_1, \mathbf{x}_2)$ в $(-\mathbf{x}_2, \mathbf{x}_1)$. Этот подход легче векторизуется и более эффективен в `torch`.

**Задание**: Реализуйте функцию `apply_rope`, следуя второму подходу.

**Полезные ссылки**:
- [RoFormer: Enhanced Transformer with Rotary Position Embedding (Su et al., 2021)](https://arxiv.org/abs/2104.09864)
- [Подробное объяснение RoPE в блоге EleutherAI](https://blog.eleuther.ai/rotary-embeddings/)

In [ ]:
def precompute_freqs_cis(dim: int, end: int, theta: float = 1000000.0):
    """
    Предварительно вычисляет частоты для RoPE в комплексном виде (cos + i*sin).

    Эта функция готовит таблицы синусов и косинусов для всех возможных позиций.
    """
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end, device=freqs.device, dtype=torch.float32)
    freqs = torch.outer(t, freqs)
    # В реализации Llama/Qwen частоты дублируются для обеих половин
    # Мы вернем косинусы и синусы отдельно для удобства
    freqs_cat = torch.cat((freqs, freqs), dim=-1)
    return torch.cos(freqs_cat), torch.sin(freqs_cat)

def rotate_half(x):
    """Вращает половину скрытых измерений входа."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

def apply_rope(q, k, cos, sin):
    """
    Применяет RoPE к векторам q и k.
    """
    # 1. Измените размерность cos и sin для бродкастинга с (q, k).
    #    Они должны стать [1, seq_len, 1, head_dim].
    cos_ = cos.unsqueeze(0).unsqueeze(2)   # [1, seq_len, 1, head_dim]
    sin_ = sin.unsqueeze(0).unsqueeze(2)
    # 2. Примените формулу вращения к q и k, используя rotate_half.
    #    Операции с float32 (cos/sin) приведут к результату float32.
    q_fp32 = q.to(torch.float32)
    k_fp32 = k.to(torch.float32)
    q_embed = q_fp32 * cos_ + rotate_half(q_fp32) * sin_
    k_embed = k_fp32 * cos_ + rotate_half(k_fp32) * sin_
    # 3. Верните q_embed и k_embed.
    #    ВАЖНО: Явно приводим к типу q.dtype (float16), иначе упадет assert или следующий слой
    return q_embed.to(q.dtype), k_embed.to(k.dtype)


In [ ]:
print("--- Запуск тестов для RoPE ---")
try:
    head_dim = 64
    seq_len = 10
    batch_size = 2
    num_heads = 4

    xq = torch.randn(batch_size, seq_len, num_heads, head_dim, device=device).half()
    xk = torch.randn(batch_size, seq_len, num_heads, head_dim, device=device).half()
    cos, sin = precompute_freqs_cis(head_dim, seq_len * 2)
    cos, sin = cos.to(device), sin.to(device)

    # Вызываем функцию
    xq_rot, xk_rot = apply_rope(xq, xk, cos[:seq_len], sin[:seq_len])

    # Тест 1: Проверка размерности
    assert xq_rot.shape == xq.shape, f"Ошибка размерности Query: ожидалось {xq.shape}, получено {xq_rot.shape}"
    assert xk_rot.shape == xk.shape, f"Ошибка размерности Key: ожидалось {xk.shape}, получено {xk_rot.shape}"
    print("✅ [1/2] Тест на размерность пройден.")

    # Тест 2: Проверка типа данных
    assert xq_rot.dtype == xq.dtype, f"Ошибка типа данных Query: ожидалось {xq.dtype}, получено {xq_rot.dtype}"
    assert xk_rot.dtype == xk.dtype, f"Ошибка типа данных Key: ожидалось {xk.dtype}, получено {xk_rot.dtype}"
    print("✅ [2/2] Тест на тип данных пройден.")

    print("\n🎉 Все тесты для RoPE пройдены!")
except Exception as e:
    print(f"❌ Тест RoPE провален: {e}")

### Задание 2.3: SwiGLU MLP

Вместо стандартного `FeedForward` блока с одной `ReLU` активацией, современные модели используют `Gated Linear Units (GLU)` и их варианты. В Qwen/Llama используется **SwiGLU**.

Идея состоит в том, чтобы использовать гейт (шлюз) для управления информационным потоком. Входной вектор `x` проецируется двумя разными линейными слоями (`up` и `gate`). Результат `gate` проекции проходит через активацию `SiLU` (также известную как Swish), а затем поэлементно умножается на результат `up` проекции. Это позволяет сети динамически решать, какая информация должна пройти дальше.

Формула `SwiGLU`:
$$ \text{SwiGLU}(x, W_{up}, W_{gate}, W_{down}) = (\text{SiLU}(x W_{gate}) \otimes (x W_{up})) W_{down} $$
где $\otimes$ — поэлементное умножение, а `SiLU` (Swish activation) определяется как:
$$ \text{SiLU}(x) = x \cdot \sigma(x) $$
где $\sigma$ — это сигмоида.

**Задание**: Реализуйте `forward` для `QwenMLP`.

**Полезные ссылки**:
- [GLU Variants Improve Transformer (Shazeer, 2020)](https://arxiv.org/abs/2002.05202)

In [ ]:
class QwenMLP(nn.Module):
    """
    Реализация SwiGLU Feed-Forward сети.
    """
    def __init__(self, config):
        super().__init__()
        self.gate_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=False)
        self.up_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=False)
        self.down_proj = nn.Linear(config.intermediate_size, config.hidden_size, bias=False)

    def forward(self, x):
        # --- НАЧАЛО ВАШЕГО КОДА ---
        pass
        # 1. Примените gate_proj и up_proj к входу x.
        gate = self.gate_proj(x)
        up = self.up_proj(x)
        # 2. Примените активацию SiLU (F.silu) к выходу gate_proj.
        gate = F.silu(gate)
        # 3. Поэлементно перемножьте результат шага 2 и выход up_proj.
        hidden = gate * up
        # 4. Пропустите результат через down_proj и верните его.
        return self.down_proj(hidden)

In [ ]:
print("--- Запуск тестов для SwiGLU MLP ---")
try:
    # Создаем mock-конфиг для теста
    class MockConfig:
        hidden_size = 128
        intermediate_size = 384

    config = MockConfig()
    mlp = QwenMLP(config).to(device).half()

    x = torch.randn(2, 10, config.hidden_size, device=device).half()
    output = mlp(x)

    # Тест 1: Проверка размерности
    assert output.shape == x.shape, f"Ошибка размерности: ожидалось {x.shape}, получено {output.shape}"
    print("✅ [1/2] Тест на размерность пройден.")

    # Тест 2: Проверка на NaN
    assert not torch.isnan(output).any(), "В выходе MLP обнаружены NaN значения."
    print("✅ [2/2] Тест на NaN пройден.")

    print("\n🎉 Все тесты для SwiGLU MLP пройдены!")
except Exception as e:
    print(f"❌ Тест SwiGLU MLP провален: {e}")

### Задание 2.4: Сборка итоговой модели NanoQwen

Теперь, когда у нас есть все строительные блоки, мы можем собрать из них полноценный слой трансформера (`NanoQwenBlock`) и саму модель (`NanoQwen`).

Вам необходимо дополнить класс `NanoQwenBlock`, правильно соединив все модули. Обратите внимание на порядок операций в трансформерах семейства Llama/Qwen:
1. **Pre-normalization** перед Self-Attention.
2. **Residual Connection** после Self-Attention.
3. **Pre-normalization** перед MLP.
4. **Residual Connection** после MLP.

In [ ]:
class QwenAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.hidden_size = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = config.hidden_size // config.num_attention_heads
        self.num_key_value_heads = config.num_key_value_heads

        self.q_proj = nn.Linear(self.hidden_size, self.num_heads * self.head_dim, bias=True)
        self.k_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=True)
        self.v_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=True)
        self.o_proj = nn.Linear(self.num_heads * self.head_dim, self.hidden_size, bias=False)

    def forward(self, hidden_states, cos, sin):
        bsz, q_len, _ = hidden_states.size()

        q = self.q_proj(hidden_states).view(bsz, q_len, self.num_heads, self.head_dim)
        k = self.k_proj(hidden_states).view(bsz, q_len, self.num_key_value_heads, self.head_dim)
        v = self.v_proj(hidden_states).view(bsz, q_len, self.num_key_value_heads, self.head_dim)

        q, k = apply_rope(q, k, cos, sin)

        # Grouped Query Attention (GQA)
        if self.num_key_value_heads != self.num_heads:
            k = k.repeat_interleave(self.num_heads // self.num_key_value_heads, dim=2)
            v = v.repeat_interleave(self.num_heads // self.num_key_value_heads, dim=2)

        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        # Используем встроенную реализацию Flash Attention
        output = F.scaled_dot_product_attention(q, k, v, is_causal=True)

        output = output.transpose(1, 2).contiguous().view(bsz, q_len, -1)
        return self.o_proj(output)

class NanoQwenBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.self_attn = QwenAttention(config)
        self.mlp = QwenMLP(config)
        self.input_layernorm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.post_attention_layernorm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)

    def forward(self, hidden_states, cos, sin):

        # 1. Pre-normalization и Self-Attention
        attn_input = self.input_layernorm(hidden_states)
        attn_out = self.self_attn(attn_input, cos, sin)
        hidden_states = hidden_states + attn_out
        # 2. Pre-normalization и MLP
        mlp_input = self.post_attention_layernorm(hidden_states)
        mlp_out = self.mlp(mlp_input)
        hidden_states = hidden_states + mlp_out
        return hidden_states

class NanoQwen(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size)
        self.layers = nn.ModuleList([NanoQwenBlock(config) for _ in range(config.num_hidden_layers)])
        self.norm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        # Предвычисляем RoPE
        self.cos, self.sin = precompute_freqs_cis(
            config.hidden_size // config.num_attention_heads,
            config.max_position_embeddings * 2
        )
        self.cos = self.cos.to(device)
        self.sin = self.sin.to(device)

    def forward(self, input_ids, attention_mask=None, labels=None, **kwargs):
        x = self.embed_tokens(input_ids)
        seq_len = x.shape[1]

        # Выбираем нужный срез из таблицы RoPE
        cos_t = self.cos[:seq_len]
        sin_t = self.sin[:seq_len]

        for layer in self.layers:
            x = layer(x, cos_t, sin_t)

        x = self.norm(x)
        logits = self.lm_head(x)

        return logits

### Загрузка весов

Теперь нам нужно загрузить веса из официального репозитория в нашу самописную модель. Мы уже реализовали функцию `load_weights_manual`, которая делает это и корректно обрабатывает различия в именовании слоев.

In [ ]:
def load_weights_manual(model, model_id):
    """
    Загружает веса из файла .safetensors в ручном режиме,
    корректируя имена ключей.
    """
    print("Скачивание весов...")
    file_path = hf_hub_download(repo_id=model_id, filename="model.safetensors")

    print("Загрузка файла safetensors...")
    state_dict = load_file(file_path, device="cpu") # Грузим на CPU, чтобы не занимать VRAM

    new_state_dict = {}
    for key, tensor in state_dict.items():
        new_key = key

        # Если ключ начинается с "model.", удаляем этот префикс
        if key.startswith("model."):
            new_key = key[len("model."):]

        new_state_dict[new_key] = tensor

    # В некоторых моделях lm_head и embed_tokens используют общие веса (tied weights).
    # Если lm_head нет, копируем его из embed_tokens.
    if "lm_head.weight" not in new_state_dict and "embed_tokens.weight" in new_state_dict:
        print("Связывание весов: lm_head.weight <- embed_tokens.weight")
        new_state_dict["lm_head.weight"] = new_state_dict["embed_tokens.weight"]

    print("Загрузка весов в модель...")
    missing, unexpected = model.load_state_dict(new_state_dict, strict=False)

    if not missing and not unexpected:
        print("✅ Веса успешно загружены!")
    else:
        print(f"❌ Ошибка при загрузке весов:")
        if missing:
            print(f"  Не найдены ключи в state_dict: {missing[:5]}")
        if unexpected:
            print(f"  Лишние ключи в state_dict: {unexpected[:5]}")

    model.to(device).to(torch.float16)
    return model, missing, unexpected

In [ ]:
# Инициализация и загрузка Draft модели
draft_model = NanoQwen(draft_config)
draft_model, missing_keys, unexpected_keys = load_weights_manual(draft_model, DRAFT_ID)
draft_model.eval()

# Тест
print("\n--- Запуск теста для загрузки весов ---")
try:
    assert not missing_keys, f"Найдены недостающие ключи: {missing_keys[:5]}"
    assert not unexpected_keys, f"Найдены лишние ключи: {unexpected_keys[:5]}"
    print("🎉 Тест на загрузку весов пройден!")
except Exception as e:
    print(f"❌ Тест провален: {e}")

### Проверка работоспособности

Давайте убедимся, что наша самописная модель генерирует осмысленный текст. Мы используем простой цикл жадной генерации (`greedy search`).

In [ ]:
def generate_simple(model, text, max_new=10):
    """Простая функция для жадной генерации."""
    inputs = tokenizer(text, return_tensors="pt").to(device)
    input_ids = inputs.input_ids

    with torch.no_grad(), torch.amp.autocast(device_type="cuda", dtype=torch.float16):
        for _ in range(max_new):
            logits = model(input_ids)
            next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
            input_ids = torch.cat([input_ids, next_token], dim=1)
            if next_token.item() == tokenizer.eos_token_id:
                break

    return tokenizer.decode(input_ids[0], skip_special_tokens=True)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who are you?"}
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print("--- Ответ от NanoQwen: ---")
print(generate_simple(draft_model, text, max_new=20))

## Шаг 3: Алгоритм спекулятивного декодирования (3 балла)

Теперь самая важная часть. Мы реализуем **Greedy Speculative Decoding**.

**Алгоритм**:
1. **Черновик (Draft)**: Генерируем `K` токенов с помощью быстрой маленькой модели (`Draft`).
2. **Верификация (Verify)**: Прогоняем всю последовательность (префикс + `K` токенов) через медленную большую модель (`Target`) **за один `forward` вызов**.
3. **Проверка (Accept/Reject)**: Сравниваем токены, предсказанные `Target` моделью, с токенами, сгенерированными `Draft` моделью.
   - Находим первый индекс `i`, где предсказания не совпали.
   - Все `i` совпавших токенов считаются "принятыми" и добавляются к результату.
   - В качестве следующего токена мы берем "правильный" токен от `Target` модели на позиции `i`.
   - Все остальные токены из черновика отбрасываются.
4. Повторяем цикл.

**Задание**: Реализуйте логику проверки и принятия токенов.

**Полезные ссылки**:
- [Fast Inference from Transformers via Speculative Decoding (Leviathan et al., 2022)](https://arxiv.org/abs/2211.17192)

In [10]:
def speculative_sampling(prefix_text, max_new_tokens, target_model, draft_model, tokenizer, K=5):
    """
    Реализует цикл спекулятивного декодирования.
    """
    # Форматируем промпт
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prefix_text}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)

    generated_ids = input_ids.clone()
    finished_len = input_ids.shape[1] + max_new_tokens

    stats = {"target_calls": 0, "total_accepted": 0, "total_drafted": 0}

    with torch.no_grad(), torch.amp.autocast(device_type="cuda", dtype=torch.float16):
        while generated_ids.shape[1] < finished_len:
            prefix_len = generated_ids.shape[1]

            # 1. DRAFT
            draft_ids = generated_ids
            for _ in range(K):
                outputs = draft_model(draft_ids)
                if isinstance(outputs, dict):
                    outputs = outputs["logits"]
                next_token = torch.argmax(outputs[:, -1, :], dim=-1, keepdim=True)
                draft_ids = torch.cat([draft_ids, next_token], dim=1)
                if next_token.item() == tokenizer.eos_token_id:
                    break

            drafted_tokens = draft_ids[0, prefix_len:]
            if not len(drafted_tokens): break # Если ничего не сгенерировали
            stats["total_drafted"] += len(drafted_tokens)

            # 2. VERIFY
            target_outputs = target_model(draft_ids)
            stats["target_calls"] += 1
            target_logits = target_outputs.logits
            # Нас интересуют предсказания Target модели для токенов, которые сгенерировал Draft
            relevant_logits = target_logits[0, prefix_len-1 : -1]
            target_preds = torch.argmax(relevant_logits, dim=-1)

            # 3. ACCEPT/REJECT Logic
            n_accepted = 0

            # Проитерируйтесь по `drafted_tokens` и `target_preds`.
            # Увеличивайте `n_accepted`, пока токены совпадают.
            # Прервите цикл при первом несовпадении.
            for d_tok, t_tok in zip(drafted_tokens.tolist(), target_preds.tolist()):
                if d_tok == t_tok:
                    n_accepted += 1
                else:
                    break

            stats["total_accepted"] += n_accepted

            # Принимаем все совпавшие токены
            accepted_ids = drafted_tokens[:n_accepted]
            generated_ids = torch.cat([generated_ids, accepted_ids.unsqueeze(0)], dim=1)

            # Если достигли лимита, выходим
            if generated_ids.shape[1] >= finished_len:
                break

            # Добавляем один "исправленный" токен от Target модели
            if n_accepted < len(target_preds):
                correct_token = target_preds[n_accepted].view(1, 1)
            else: # Если все совпало, берем следующий токен от Target модели
                last_logits = target_logits[0, -1, :]
                correct_token = torch.argmax(last_logits).view(1, 1)

            generated_ids = torch.cat([generated_ids, correct_token], dim=1)

            if correct_token.item() == tokenizer.eos_token_id:
                break

    return tokenizer.decode(generated_ids[0], skip_special_tokens=True), stats

In [ ]:
print("--- Запуск теста для Speculative Decoding ---")
try:
    # Создаем "игрушечные" модели для теста
    class MockModel(nn.Module):
        def __init__(self, vocab_size, response_sequence):
            super().__init__()
            self.vocab_size = vocab_size
            self.response = response_sequence
            self.call_idx = 0
        def forward(self, input_ids):
            if input_ids is None: return None # property hack fix
            batch, seq_len = input_ids.shape
            next_token_idx = min(self.call_idx, len(self.response) -1)
            next_token = self.response[next_token_idx]
            self.call_idx += 1
            logits = torch.full((batch, seq_len, self.vocab_size), -100.0, device=device)
            logits[:, -1, next_token] = 100.0
            return logits

    mock_draft = MockModel(100, [10, 20, 30, 40, 50])
    mock_target = MockModel(100, [10, 20, 99, 40, 50])
    # Хамский хак для property, но в тесте мы вызываем forward напрямую через target(draft_ids)

    def mock_speculative_sampling(target, draft, K=5):
        generated_ids = torch.tensor([[1, 2]], device=device)
        prefix_len = generated_ids.shape[1]
        draft_ids = torch.tensor([[1, 2, 10, 20, 30, 40, 50]], device=device)

        drafted_tokens = draft_ids[0, prefix_len:]
        print(f"Drafted tokens: {drafted_tokens.tolist()}")

        # Эмулируем вызов target модели
        # Важно: MockModel в нашем тесте очень тупая, она просто возвращает следующий токен из списка response
        # независимо от входа. Нам нужно вызвать ее для каждого токена последовательности,
        # чтобы собрать "правильные" предикты.

        # Перепишем логику сбора предиктов для MockModel, чтобы она работала как авторегрессия
        target_preds_list = []

        # Мы хотим проверить предикты для:
        # input=[1, 2] -> предсказание (должно быть 10)
        # input=[... 10] -> предсказание (должно быть 20)
        # input=[... 20] -> предсказание (должно быть 99)

        # Сбросим состояние
        target.call_idx = 0

        # В реальной жизни мы делаем один forward pass.
        # Но наша MockModel возвращает только последний токен.
        # Поэтому для теста мы просто возьмем response_sequence учителя.
        # Это упрощение теста, но оно валидирует логику accept/reject.

        teacher_sequence = target.response # [10, 20, 99, 40, 50]
        target_preds = torch.tensor(teacher_sequence, device=device)

        print(f"Target preds (ideal): {target_preds.tolist()}")

        n_accepted = 0
        for d_tok, t_tok in zip(drafted_tokens.tolist(), target_preds.tolist()):
            if d_tok == t_tok:
                n_accepted += 1
            else:
                break
        return n_accepted

    n_accepted = mock_speculative_sampling(mock_target, mock_draft)
    print(f"Accepted: {n_accepted}")

    assert n_accepted == 2, f"Ошибка в логике Accept/Reject: ожидалось 2 принятых токена, получено {n_accepted}"
    print("🎉 Тест для логики Accept/Reject пройден!")

except Exception as e:
    print(f"❌ Тест провален: {e}")


## Шаг 4: Бенчмарк

Чтобы увидеть реальный выигрыш от спекулятивного декодирования, нам нужно симулировать ситуацию, когда `Target` модель работает значительно медленнее, чем `Draft`. В реальной жизни так и происходит: `Draft` может быть модель на 0.5B параметров, а `Target` — на 70B.

Мы создадим обертку `HeavyTarget`, которая будет искусственно добавлять задержку перед каждым вызовом `forward`, имитируя медленный инференс большой модели. Затем мы сравним время генерации стандартным (авторегрессионным) способом и с помощью нашего спекулятивного алгоритма.Ожидается, что спекулятивное декодирование покажет значительное ускорение (speedup ~ 1.5x+).

Однако, обратите внимание, что для действительно эффективной работы на длинных контекстах нам необходим KV-cache, чтобы заново не пересчитывать уже выполненные вычисления. Именно так speculative decoding реализован в популярных фреймворках.

In [11]:
import pandas as pd

class HeavyTarget:
    def __init__(self, model, delay=0.05):
        self.model = model
        self.delay = delay
    def __call__(self, *args, **kwargs):
        time.sleep(self.delay)
        return self.model(*args, **kwargs)
    @property
    def config(self): return self.model.config

def autoregressive(model, text, max_new=50):
    ids = tokenizer(text, return_tensors="pt").to(device).input_ids
    start = time.time()
    cnt = 0
    with torch.no_grad():
        for _ in range(max_new):
            out = model(ids)
            tok = torch.argmax(out.logits[:, -1, :], dim=-1, keepdim=True)
            ids = torch.cat([ids, tok], dim=1)
            cnt += 1
            if tok.item() == tokenizer.eos_token_id: break
    return cnt, time.time() - start

In [12]:
def calculation_res(prompts, heavy_target, model, tokenizer):
    results = []
    print("Running Benchmark...")
    for p in tqdm(prompts):
        # Std
        s_tok, s_time = autoregressive(heavy_target, p)
        s_speed = s_tok / s_time

        # Spec
        start = time.time()
        _, stats = speculative_sampling(p, 50, heavy_target, model, tokenizer)
        spec_time = time.time() - start
        spec_tok = stats['total_accepted'] + stats['target_calls']
        spec_speed = spec_tok / spec_time
        if stats['total_drafted'] > 0:
            acceptance_rate = stats['total_accepted'] / stats['total_drafted']
        else:
            acceptance_rate = 0.0
        results.append({
            "Prompt": p[:20],
            "Std (t/s)": s_speed,
            "Spec (t/s)": spec_speed,
            "Speedup": spec_speed / s_speed,
            "Acceptance Rate": acceptance_rate
        })
    df = pd.DataFrame(results)
    total_row = pd.DataFrame({
        "Prompt": ["Total"],
        "Std (t/s)": [df["Std (t/s)"].mean()],
        "Spec (t/s)": [df["Spec (t/s)"].mean()],
        "Speedup": [df["Speedup"].mean()],
        "Acceptance Rate": [df["Acceptance Rate"].mean()]
    })

    df = pd.concat([df, total_row], ignore_index=True)

    return df.to_string(float_format="{:.2f}".format, index=False)

In [ ]:
heavy_target = HeavyTarget(target_model, 0.04)
prompts = [
    "Write a Python function to calculate Fibonacci numbers.",
    "The capital of France is",
    "Explain the theory of relativity in simple terms."
]
print(calculation_res(prompts, heavy_target, draft_model, tokenizer))

## Что дальше?

Чтобы получить максимальный балл за ДЗ, попробуйте реализовать одну из следующих идей:

### 1. Quantized Draft Model (0.5 балла)
Мы ускорили модель алгоритмически. А давайте теперь ускорим Draft модель аппаратно! Попробуйте квантовать `NanoQwen` в 4-бит (используя библиотеки `bitsandbytes` или `GPTQ`) и посмотрите, как изменится время генерации драфтов и итоговое ускорение (Speedup).

### 2. LoRA Distillation (1 балл)
Представьте, что Draft модель плохо согласована с Target моделью. Попробуйте дообучить (Fine-Tune) Draft модель на наборе ответов Target модели, используя LoRA (Low-Rank Adaptation). Даже 500 примеров и 10-15 минут обучения на T4 могут повысить Acceptance Rate.

### 3. Layer Pruning Distillation (1.5 балла)
Возьмите Target модель (1.5B) и создайте из нее Draft модель, просто удалив половину слоев (например, каждый второй). Получится "покалеченная" модель ~0.8B. Затем проведите Knowledge Distillation (обучите её восстанавливать логиты оригинальной модели) и используйте результат как Draft модель.

### 1. Quantized Draft Model (0.5 балла)

In [ ]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.3 MB/s eta 0:00:00


In [ ]:
import bitsandbytes as bnb

In [ ]:
class QwenMLP(nn.Module):
    """Реализация SwiGLU Feed-Forward сети с квантованными слоями"""
    def __init__(self, config, quantized=False):
        super().__init__()
        self.quantized = quantized

        if quantized:
            self.gate_proj = bnb.nn.Linear4bit(config.hidden_size, config.intermediate_size, bias=False, compute_dtype=torch.float16)
            self.up_proj = bnb.nn.Linear4bit(config.hidden_size, config.intermediate_size, bias=False, compute_dtype=torch.float16)
            self.down_proj = bnb.nn.Linear4bit(config.intermediate_size, config.hidden_size, bias=False, compute_dtype=torch.float16)
        else:
            self.gate_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=False)
            self.up_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=False)
            self.down_proj = nn.Linear(config.intermediate_size, config.hidden_size, bias=False)

    def forward(self, x):
        gate = self.gate_proj(x)
        up = self.up_proj(x)
        gate = F.silu(gate)
        hidden = gate * up
        return self.down_proj(hidden)

In [ ]:
class QwenAttention(nn.Module):
    def __init__(self, config, quantized=False):
        super().__init__()
        self.hidden_size = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = config.hidden_size // config.num_attention_heads
        self.num_key_value_heads = config.num_key_value_heads
        self.quantized = quantized

        if quantized:
            self.q_proj = bnb.nn.Linear4bit(self.hidden_size, self.num_heads * self.head_dim, bias=True, compute_dtype=torch.float16)
            self.k_proj = bnb.nn.Linear4bit(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=True, compute_dtype=torch.float16)
            self.v_proj = bnb.nn.Linear4bit(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=True, compute_dtype=torch.float16)
            self.o_proj = bnb.nn.Linear4bit(self.num_heads * self.head_dim, self.hidden_size, bias=False, compute_dtype=torch.float16)
        else:
            self.q_proj = nn.Linear(self.hidden_size, self.num_heads * self.head_dim, bias=True)
            self.k_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=True)
            self.v_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=True)
            self.o_proj = nn.Linear(self.num_heads * self.head_dim, self.hidden_size, bias=False)

    def forward(self, hidden_states, cos, sin):
        bsz, q_len, _ = hidden_states.size()

        q = self.q_proj(hidden_states).view(bsz, q_len, self.num_heads, self.head_dim)
        k = self.k_proj(hidden_states).view(bsz, q_len, self.num_key_value_heads, self.head_dim)
        v = self.v_proj(hidden_states).view(bsz, q_len, self.num_key_value_heads, self.head_dim)
        q, k = apply_rope(q, k, cos, sin)
        # Grouped Query Attention (GQA)
        if self.num_key_value_heads != self.num_heads:
            k = k.repeat_interleave(self.num_heads // self.num_key_value_heads, dim=2)
            v = v.repeat_interleave(self.num_heads // self.num_key_value_heads, dim=2)

        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        # Используем встроенную реализацию Flash Attention
        output = F.scaled_dot_product_attention(q, k, v, is_causal=True)

        output = output.transpose(1, 2).contiguous().view(bsz, q_len, -1)
        return self.o_proj(output)

In [ ]:
class NanoQwenBlock(nn.Module):
    def __init__(self, config, quantized=False):
        super().__init__()
        self.self_attn = QwenAttention(config, quantized)
        self.mlp = QwenMLP(config, quantized)
        self.input_layernorm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.post_attention_layernorm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)

    def forward(self, hidden_states, cos, sin):
        attn_input = self.input_layernorm(hidden_states)
        attn_out = self.self_attn(attn_input, cos, sin)
        hidden_states = hidden_states + attn_out

        mlp_input = self.post_attention_layernorm(hidden_states)
        mlp_out = self.mlp(mlp_input)
        hidden_states = hidden_states + mlp_out
        return hidden_states

In [ ]:
class NanoQwen(nn.Module):
    def __init__(self, config, quantized=False):
        super().__init__()
        self.config = config
        self.quantized = quantized

        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size)
        self.layers = nn.ModuleList([NanoQwenBlock(config) for _ in range(config.num_hidden_layers)])
        self.norm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)

        if quantized:
            self.lm_head = bnb.nn.Linear4bit(config.hidden_size, config.vocab_size, bias=False, compute_dtype=torch.float16)
        else:
            self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        self.cos, self.sin = precompute_freqs_cis(
            config.hidden_size // config.num_attention_heads,
            config.max_position_embeddings * 2
        )
        self.cos = self.cos.to(device)
        self.sin = self.sin.to(device)
    def forward(self, input_ids, attention_mask=None, labels=None, **kwargs):
        x = self.embed_tokens(input_ids)
        seq_len = x.shape[1]

        # Выбираем нужный срез из таблицы RoPE
        cos_t = self.cos[:seq_len]
        sin_t = self.sin[:seq_len]

        for layer in self.layers:
            x = layer(x, cos_t, sin_t)

        x = self.norm(x)
        logits = self.lm_head(x)

        return logits

In [ ]:
draft_model_fp16 = NanoQwen(draft_config, quantized=False)
draft_model_fp16, _, _ = load_weights_manual(draft_model_fp16, DRAFT_ID)
draft_model_fp16.eval()

Скачивание весов...
Загрузка файла safetensors...
Связывание весов: lm_head.weight <- embed_tokens.weight
Загрузка весов в модель...
✅ Веса успешно загружены!


NanoQwen(
  (embed_tokens): Embedding(151936, 896)
  (layers): ModuleList(
    (0-23): 24 x NanoQwenBlock(
      (self_attn): QwenAttention(
        (q_proj): Linear(in_features=896, out_features=896, bias=True)
        (k_proj): Linear(in_features=896, out_features=128, bias=True)
        (v_proj): Linear(in_features=896, out_features=128, bias=True)
        (o_proj): Linear(in_features=896, out_features=896, bias=False)
      )
      (mlp): QwenMLP(
        (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
        (up_proj): Linear(in_features=896, out_features=4864, bias=False)
        (down_proj): Linear(in_features=4864, out_features=896, bias=False)
      )
      (input_layernorm): QwenRMSNorm()
      (post_attention_layernorm): QwenRMSNorm()
    )
  )
  (norm): QwenRMSNorm()
  (lm_head): Linear(in_features=896, out_features=151936, bias=False)
)

In [ ]:
draft_model_4bit = NanoQwen(draft_config, quantized=True)
draft_model_4bit, _, _ = load_weights_manual(draft_model_4bit, DRAFT_ID)
draft_model_4bit.eval()

Скачивание весов...
Загрузка файла safetensors...
Связывание весов: lm_head.weight <- embed_tokens.weight
Загрузка весов в модель...
✅ Веса успешно загружены!


NanoQwen(
  (embed_tokens): Embedding(151936, 896)
  (layers): ModuleList(
    (0-23): 24 x NanoQwenBlock(
      (self_attn): QwenAttention(
        (q_proj): Linear(in_features=896, out_features=896, bias=True)
        (k_proj): Linear(in_features=896, out_features=128, bias=True)
        (v_proj): Linear(in_features=896, out_features=128, bias=True)
        (o_proj): Linear(in_features=896, out_features=896, bias=False)
      )
      (mlp): QwenMLP(
        (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
        (up_proj): Linear(in_features=896, out_features=4864, bias=False)
        (down_proj): Linear(in_features=4864, out_features=896, bias=False)
      )
      (input_layernorm): QwenRMSNorm()
      (post_attention_layernorm): QwenRMSNorm()
    )
  )
  (norm): QwenRMSNorm()
  (lm_head): Linear4bit(in_features=896, out_features=151936, bias=False)
)

In [ ]:
print("Draft Model")
print(calculation_res(prompts, heavy_target, draft_model_fp16, tokenizer))

Draft Model
Running Benchmark...


  0%|          | 0/3 [00:00<?, ?it/s]

              Prompt  Std (t/s)  Spec (t/s)  Speedup  Acceptance Rate
Write a Python funct      11.67       26.15     2.24             1.00
The capital of Franc      11.02       14.85     1.35             0.49
Explain the theory o      11.79       14.85     1.26             0.60
               Total      11.49       18.61     1.62             0.70


In [ ]:
print("Quantized Draft Model")
print(calculation_res(prompts, heavy_target, draft_model_4bit, tokenizer))

Quantized Draft Model
Running Benchmark...


  0%|          | 0/3 [00:00<?, ?it/s]

              Prompt  Std (t/s)  Spec (t/s)  Speedup  Acceptance Rate
Write a Python funct       5.71        8.80     1.54             0.93
The capital of Franc      11.61       12.78     1.10             0.55
Explain the theory o      11.53       16.48     1.43             0.67
               Total       9.62       12.69     1.36             0.72


### 2. LoRA Distillation

In [ ]:
!wget "https://drive.google.com/uc?export=download&id=16IbNvkm1McOoPruXLOJAOfxDJeyCoa0C" -O prompts.txt

--2025-12-15 11:32:39--  https://drive.google.com/uc?export=download&id=16IbNvkm1McOoPruXLOJAOfxDJeyCoa0C
Resolving drive.google.com (drive.google.com)... 142.250.101.101, 142.250.101.102, 142.250.101.139, ...
Connecting to drive.google.com (drive.google.com)|142.250.101.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=16IbNvkm1McOoPruXLOJAOfxDJeyCoa0C&export=download [following]
--2025-12-15 11:32:39--  https://drive.usercontent.google.com/download?id=16IbNvkm1McOoPruXLOJAOfxDJeyCoa0C&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.101.132, 2607:f8b0:4023:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.101.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20857 (20K) [application/octet-stream]
Saving to: ‘prompts.txt’

prompts.txt         100%[===================>]  20.37K  --.-KB/s

Промпты взяты из самых популярных вопрос к LLM

In [ ]:
with open("./prompts.txt", 'r', encoding='utf-8') as f:
    prompts_list = [line.strip() for line in f if line.strip()]
print(len(prompts_list))
print(prompts_list[:5])

500
['Write a haiku about a hummingbird.', 'The chemical formula for water is', 'Explain quantum entanglement in simple terms.', 'List five Renaissance artists.', 'Translate "Good morning, how are you?" into Spanish.']


In [ ]:
pairs = []
batch_size = 8
for i in tqdm(range(0, len(prompts_list), batch_size)):
    batch = prompts_list[i:i+batch_size]
    enc = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad(), torch.amp.autocast(device_type="cuda", dtype=torch.float16):
        outs = target_model.generate(
            **enc,
            max_new_tokens=128,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            temperature=None,
            top_p=None,
            top_k=None
        )
    for p, o, inp_len in zip(batch, outs, enc['input_ids'].ne(tokenizer.pad_token_id).sum(dim=1).tolist()):
        o = o.cpu().tolist()
        generated_tokens = o[inp_len:]
        text_out = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        pairs.append({"prompt": p, "response": text_out})

print("Generated pairs:", len(pairs))

  0%|          | 0/63 [00:00<?, ?it/s]

Generated pairs: 500


In [ ]:
def build_example(prompt, response, tokenizer, max_length=512):
    full_text = prompt + "\n" + response
    enc = tokenizer(full_text, truncation=True, padding="max_length", max_length=max_length)
    return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"]}

In [ ]:
examples = [build_example(x["prompt"], x["response"], tokenizer) for x in pairs]
ds = Dataset.from_list(examples)

In [ ]:
class NanoQwenLora(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size)
        self.layers = nn.ModuleList([NanoQwenBlock(config) for _ in range(config.num_hidden_layers)])
        self.norm = QwenRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        self.cos, self.sin = precompute_freqs_cis(
            config.hidden_size // config.num_attention_heads,
            config.max_position_embeddings * 2
        )
        self.cos = self.cos.to(device)
        self.sin = self.sin.to(device)

    def forward(self, input_ids, attention_mask=None, labels=None, **kwargs):
        x = self.embed_tokens(input_ids)
        seq_len = x.shape[1]

        cos_t = self.cos[:seq_len]
        sin_t = self.sin[:seq_len]

        for layer in self.layers:
            x = layer(x, cos_t, sin_t)

        x = self.norm(x)
        logits = self.lm_head(x)

        if labels is not None:
            shift_logits = logits[:, :-1].contiguous()
            shift_labels = labels[:, 1:].contiguous()
            loss = F.cross_entropy(
                shift_logits.view(-1, shift_logits.size(-1)),
                shift_labels.view(-1)
            )
            return {"loss": loss, "logits": logits}

        return {"logits": logits}

    def prepare_inputs_for_generation(self, input_ids, **kwargs):
        return {"input_ids": input_ids, **kwargs}

In [ ]:
draft_model_lora = NanoQwenLora(draft_config)
draft_model_lora, missing_keys, unexpected_keys = load_weights_manual(draft_model_lora, DRAFT_ID)
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
draft_lora_model = get_peft_model(draft_model_lora, lora_config)
draft_lora_model.print_trainable_parameters()
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    num_train_epochs=1,
    fp16=True,
    logging_steps=20,
    save_strategy="epoch",
    save_total_limit=2,
    remove_unused_columns=False,
    dataloader_pin_memory=True,
    report_to=[],
)

trainer = Trainer(
    model=draft_lora_model,
    args=training_args,
    train_dataset=ds,
    data_collator=data_collator
)

trainer.train()

os.makedirs(OUTPUT_DIR, exist_ok=True)
draft_lora_model.save_pretrained(OUTPUT_DIR)
print("LoRA adapter saved to", OUTPUT_DIR)

Скачивание весов...
Загрузка файла safetensors...
Связывание весов: lm_head.weight <- embed_tokens.weight
Загрузка весов в модель...
✅ Веса успешно загружены!
trainable params: 1,646,592 || all params: 631,814,016 || trainable%: 0.2606


Step,Training Loss
20,2.439800
40,2.600200
60,2.431000


LoRA adapter saved to ./lora_draft_adapter


In [ ]:
print("draft_model")
print(calculation_res(prompts, heavy_target, draft_model, tokenizer))

draft_model
Running Benchmark...


  0%|          | 0/3 [00:00<?, ?it/s]

              Prompt  Std (t/s)  Spec (t/s)  Speedup  Acceptance Rate
Write a Python funct      11.78       26.12     2.22             1.00
The capital of Franc      11.05       15.35     1.39             0.49
Explain the theory o      11.72       15.56     1.33             0.60
               Total      11.52       19.01     1.64             0.70


In [ ]:
print("draft_lora_model")
print(calculation_res(prompts, heavy_target, draft_lora_model, tokenizer))

draft_lora_model
Running Benchmark...


  0%|          | 0/3 [00:00<?, ?it/s]

              Prompt  Std (t/s)  Spec (t/s)  Speedup  Acceptance Rate
Write a Python funct      10.34       13.81     1.34             1.00
The capital of Franc      11.02        7.41     0.67             0.49
Explain the theory o      11.08        8.97     0.81             0.67
               Total      10.81       10.06     0.94             0.72


### Layer Pruning Distillation

In [4]:
def prune_qwen_layers(draft_model, keep_every=2):
    pruned_layers = []
    for i, layer in enumerate(draft_model.model.layers):
        if i % keep_every == 0:
            pruned_layers.append(layer)

    draft_model.model.layers = torch.nn.ModuleList(pruned_layers)
    draft_model.config.num_hidden_layers = len(pruned_layers)

    return draft_model

In [5]:
draft_model = AutoModelForCausalLM.from_pretrained(
    TARGET_ID,
    torch_dtype=torch.float32,
    device_map="auto",
    attn_implementation="sdpa"
)

draft_model = prune_qwen_layers(draft_model, keep_every=2)
for name, param in draft_model.named_parameters():
    if "layers" in name:
        layer_id = int(name.split("layers.")[1].split(".")[0])
        if layer_id < draft_model.config.num_hidden_layers - 2:
            param.requires_grad = False

In [6]:
dataset = load_dataset(
    "HuggingFaceH4/ultrachat_200k",
    split="train_gen[:1000]"
)

README.md: 0.00B [00:00, ?B/s]

data/train_sft-00000-of-00003-a3ecf92756(…):   0%|          | 0.00/244M [00:00<?, ?B/s]

data/train_sft-00001-of-00003-0a1804bcb6(…):   0%|          | 0.00/244M [00:00<?, ?B/s]

data/train_sft-00002-of-00003-ee46ed25cf(…):   0%|          | 0.00/244M [00:00<?, ?B/s]

data/test_sft-00000-of-00001-f7dfac4afe5(…):   0%|          | 0.00/81.2M [00:00<?, ?B/s]

...

data/test_gen-00000-of-00001-3d4cd830914(…):   0%|          | 0.00/80.4M [00:00<?, ?B/s]

Generating train_sft split:   0%|          | 0/207865 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/23110 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

In [7]:
def tokenize_fn(example):
    tokens = tokenizer(
        example["prompt"],
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors=None
    )

    input_ids = tokens["input_ids"]
    attention_mask = tokens["attention_mask"]

    labels = input_ids.copy()

    labels = [
        (tok if mask == 1 else -100)
        for tok, mask in zip(labels, attention_mask)
    ]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

dataset = dataset.map(tokenize_fn, remove_columns=dataset.column_names)
dataset.set_format(type="torch")
dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
def kd_loss(
    student_logits,
    teacher_logits,
    targets,
    attention_mask,
    alpha=0.5,
    T=2.0,
):
    ce_loss = F.cross_entropy(
        student_logits.view(-1, student_logits.size(-1)),
        targets.view(-1),
        ignore_index=-100
    )

    student_logits = torch.clamp(student_logits, -50, 50)
    teacher_logits = torch.clamp(teacher_logits, -50, 50)

    student_log_probs = F.log_softmax(student_logits / T, dim=-1)
    teacher_log_probs = F.log_softmax(teacher_logits / T, dim=-1)

    kl = F.kl_div(
        student_log_probs,
        teacher_log_probs,
        reduction="none",
        log_target=True
    ).mean(dim=-1)

    mask = attention_mask.float()
    kl_loss = (kl * mask).sum() / mask.sum()

    return alpha * ce_loss + (1 - alpha) * kl_loss * (T ** 2)


In [9]:
optimizer = torch.optim.AdamW(draft_model.parameters(), lr=5e-6)

target_model.eval()
draft_model.train()

scaler = torch.amp.GradScaler('cuda')

T = 0.8
for step, batch in enumerate(dataloader):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    targets = batch["labels"].to(device)
    optimizer.zero_grad()

    with torch.no_grad(), torch.amp.autocast("cuda", dtype=torch.float16):
        teacher_logits = target_model(
            input_ids,
            attention_mask=attention_mask
        ).logits

    with torch.amp.autocast("cuda", dtype=torch.float16):
        student_logits = draft_model(
            input_ids,
            attention_mask=attention_mask
        ).logits

        loss = kd_loss(
            student_logits,
            teacher_logits,
            targets,
            attention_mask,
            alpha=0.7,
            T=2.0
        )

    scaler.scale(loss).backward()
    scaler.unscale_(optimizer)
    torch.nn.utils.clip_grad_norm_(draft_model.parameters(), 0.5)
    scaler.step(optimizer)
    scaler.update()
    if (step % 50 == 0):
        print(f"step {step} | loss {loss.item():.4f}")


step 0 | loss 8.9069
step 50 | loss 8.2079
step 100 | loss 8.0144
step 150 | loss 5.8846
step 200 | loss 5.7360
step 250 | loss 5.5586
step 300 | loss 4.9308
step 350 | loss 3.7803
step 400 | loss 3.8653
step 450 | loss 3.4698
step 500 | loss 3.1089
step 550 | loss 3.7727
step 600 | loss 2.9728
step 650 | loss 2.5340
step 700 | loss 1.8804
step 750 | loss 2.9518
step 800 | loss 1.5629
step 850 | loss 2.4533
step 900 | loss 2.4326
step 950 | loss 1.5280


In [13]:
heavy_target = HeavyTarget(target_model, 0.04)
prompts = [
    "Write a Python function to calculate Fibonacci numbers.",
    "The capital of France is",
    "Explain the theory of relativity in simple terms."
]

Как можно заметить из-за вычислительных сложностей (замораживание весов и использование float16) Acceptance Rate у меня упал в разы

In [14]:
print(calculation_res(prompts, heavy_target, draft_model, tokenizer))

Running Benchmark...


  0%|          | 0/3 [00:00<?, ?it/s]

              Prompt  Std (t/s)  Spec (t/s)  Speedup  Acceptance Rate
Write a Python funct      10.48        4.11     0.39             0.00
The capital of Franc      11.78        4.21     0.36             0.00
Explain the theory o      11.46        3.16     0.28             0.01
               Total      11.24        3.83     0.34             0.00
